In [ ]:
# Pseudocode Spectral Clustering
# procedure spectral_clustering(data, n_clusters, n_neighbors):
#     1. Compute the weight matrix using k-nearest neighbors, where k = n_neighbors
#     2. Compute the Laplacian matrix:
#        a. Compute the degree matrix from the weight matrix.
#        b. Compute the unnormalized Laplacian matrix as L = D - W
#     3. Compute the eigenvectors and eigenvalues of the Laplacian matrix
#     4. Select the top k eigenvectors corresponding to the smallest
#     ...eigenvalues to form a matrix U, where the columns
#     ...of U are the k eigenvectors.
#     5. Perform K-means or another clustering algorithm on the rows of U
#     ...to obtain the final clusters.
#     return clusters

In [1]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.cluster import SpectralClustering

In [11]:
class Spectral_Clustering:
	def __init__(self, data ,n_clusters, n_neighbors) :
		self.x = data
		self.n_clusters = n_clusters
		self.n_neighbors = n_neighbors
		self.num = data.shape[0]
	
	def distance(self, a, b):
		distance = np.sqrt((a[0]-b[0]) ** 2 + (a[1]-b[1]) ** 2)
		return distance
	
	def Weight_matrix(self):
		W = np.zeros((self.num, self.num))
		for i in range(self.num):
			dist = np.zeros(self.num)
			for j in range(self.num):
				dist[j] = self.distance(self.x[i], self.x[j])
			dist[i] = 1e10
			indices_of_smallest = dist.argsort()[:self.n_neighbors]
			for z in range(self.n_neighbors):
				W[i][indices_of_smallest[z]] = 1
		W = 0.5*W + 0.5*W.T
		return W
	
	def Laplacian(self):
		W = self.Weight_matrix()
		D = np.zeros((self.num, self.num))
		for i in range(self.num):
			D[i][i] = np.sum(W[i][:])
		L = D-W
		return L
	
	def Final_matrix(self):
		eigenvalues, eigenvectors = np.linalg.eig(self.Laplacian())
		sorted_indices = np.argsort(eigenvalues)
		U = eigenvectors[:, sorted_indices[:self.n_neighbors]]
		return U
	
	def Grouping(self):
		kmeans = KMeans(n_clusters=self.n_clusters)
		clusters = kmeans.fit_predict(self.Final_matrix())
		return clusters
	

In [12]:
np.random.seed(0)
num_samples = 10
num_features = 2
num_components = 3
means = np.array([[2,2], [8,3], [3,6]])
covariances = np.array([[[1,0.5], [0.5,1]], [[1,0], [0,1]], [[0.5,0], [0,0]]])
weights = np.array([0.4,0.3,0.3])

X = np.zeros((num_samples, num_features))
for i in range(num_components):
	num_samples_i = int (weights[i] * num_samples)
	X[i * num_samples_i: (i+1) * num_samples_i] = np.random.multivariate_normal(means[i], covariances[i], size=num_samples_i)
print(X)


[[ 0.27220725  0.67236446]
 [ 0.03194144  2.27283464]
 [ 0.87128628 -0.1059916 ]
 [ 7.89678115  3.4105985 ]
 [ 8.14404357  4.45427351]
 [ 8.76103773  3.12167502]
 [ 3.3138587   6.        ]
 [ 4.05647344  6.        ]
 [ 3.22137229  6.        ]
 [ 0.          0.        ]]


In [13]:
clustering = SpectralClustering(n_clusters=3, affinity='nearest_neighbors', n_neighbors=3)

labels = clustering.fit_predict(X)
print(labels)
%time

[1 1 1 2 2 2 0 0 0 1]
CPU times: user 14 µs, sys: 20 µs, total: 34 µs
Wall time: 11.9 µs


/Users/mazhiqi/anaconda3/lib/python3.11/site-packages/sklearn/manifold/_spectral_embedding.py:273: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [14]:
modified_clustering = Spectral_Clustering(X,3,3)

G = modified_clustering.Grouping()
print(G)
%time

[1 1 1 0 0 0 2 2 2 1]
CPU times: user 12 µs, sys: 5 µs, total: 17 µs
Wall time: 5.01 µs


/Users/mazhiqi/anaconda3/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
